In [26]:
import data_processing.port_n_vessels as port
import data_processing.weather as weather
import data_processing.port_tracker as tracker

import numpy as np
import sqlite3
from datetime import datetime, timedelta
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup


In [27]:
crawler_dict = port.crawler()
hourly_weather_data = weather.openmeteo_hourly()
port_calls = tracker.port_calls()
port_berthed_f1 = tracker.inport() #Another reference

c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https

In [28]:
scheduled = crawler_dict['scheduled']['df']
scheduled['Date'] = pd.to_datetime(scheduled['Date'], format='mixed',dayfirst=True)
scheduled['ETA'] = pd.to_datetime(scheduled['ETA'], format='mixed', dayfirst=True)
scheduled['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

foreseen_cargo = crawler_dict['foreseen_cargo']['df']
foreseen_cargo.drop(columns=['P'], axis=1, inplace=True)
foreseen_cargo['Arrival'] = pd.to_datetime(foreseen_cargo['Arrival'], format='mixed', dayfirst=True)
foreseen_cargo['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

foreseen_cruise = crawler_dict['foreseen_cruise']['df']
foreseen_cruise['Arrival'] = pd.to_datetime(foreseen_cruise['Arrival'], format='mixed', dayfirst=True)
foreseen_cruise['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

berthage_ships = crawler_dict['berthage_ships']['df']
berthage_ships['Arrival'] = pd.to_datetime(berthage_ships['Arrival'], format='mixed', dayfirst=True)
berthage_ships['Ship'] = berthage_ships['Ship'].str.replace(r'PROGRAMADO', '', case=False, regex=True)
berthage_ships['UPDATED_AT'] = datetime.now()

#-----------------------------------------------------------------------------#

berthed_ships = crawler_dict['berthed_ships']['df']
berthed_ships['Load'] = berthed_ships['Load'].astype(int)
berthed_ships['Unload'] = berthed_ships['Unload'].astype(int)
berthage_ships['UPDATED_AT'] = datetime.now()

#------------------------------------------------------------------------#

port_berthed_f1["DWT"] = port_berthed_f1["DWT"].replace(r'---', None)
port_berthed_f1["DWT"] = port_berthed_f1["DWT"].str.replace(r',', '')
port_berthed_f1["DWT_Unit"] = port_berthed_f1["DWT"].str.split().str[1]
port_berthed_f1["DWT"] = port_berthed_f1["DWT"].str.split().str[0]

#------------------------------------------------------------------------#

port_berthed_f1["GRT"] = port_berthed_f1["GRT"].replace(r'---', None)
port_berthed_f1["GRT"] = port_berthed_f1["GRT"].str.replace(r',', '')
port_berthed_f1["GRT_Unit"] = port_berthed_f1['GRT'].str.split().str[1]
port_berthed_f1["GRT"] = port_berthed_f1["GRT"].str.split().str[0]

In [63]:
conn = sqlite3.connect('data/raw_pvs.db')
cursor = conn.cursor()

In [71]:
cursor.execute('''

INSERT OR REPLACE INTO dim_ship (imo, mmsi, ship_name)

    SELECT imo, mmsi, ship_name FROM port_berthed_ships
    UNION
    SELECT imo, mmsi, ship_name FROM port_calls


''')

In [21]:
hourly_weather_data.to_sql('weather_data', conn, if_exists='append', index=False)

216

In [5]:
port_calls.to_sql('port_calls', conn, if_exists='append', index=False)

92

In [76]:
port_berthed_f1.to_sql('port_berthed_ships', conn, if_exists='append', index=False)

89

In [44]:
scheduled.to_sql('scheduled', conn, if_exists='append',index=False)

59

In [52]:
foreseen_cargo.to_sql('foreseen_cargo', conn, if_exists='append',index=False)

277

In [62]:
foreseen_cruise.to_sql('foreseen_cruise', conn, if_exists='append',index=False)

106

In [73]:
berthage_ships.to_sql('berthage_ships', conn, if_exists='append',index=False)

47

In [79]:
berthed_ships.to_sql('berthed_ships', conn, if_exists='append',index=False)

82

In [73]:
conn.commit()
conn.close()